In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import statsmodels.stats.api as sms
from statsmodels.stats.proportion import proportions_ztest
from tqdm.notebook import tqdm

In [2]:
def get_sample_size(p1, p2, alpha=0.05, power=0.8, r=1):
    p_bar = (p1 + r * p2)/(r + 1)
    q_bar = 1 - p_bar
    s = np.abs(p1 - p2)

    q1 = 1 - p1
    q2 = 1 - p2

    z_alpha = norm.ppf(1 - alpha/2)
    z_beta = norm.ppf(power)

    m = ((z_alpha * np.sqrt((r + 1) * p_bar * q_bar) +
        z_beta * np.sqrt(r * p1 * q1 + p2 * q2))**2/
        (r * s ** 2))

    return m,  m * r

Возьмем например базовый уровень 0.2 и разницу в 5%

In [3]:
get_sample_size(0.20,0.25)

(1093.7390457661654, 1093.7390457661654)

Теперь еще посчитаем функцией из statmodels. https://www.statsmodels.org/stable/generated/statsmodels.stats.power.tt_ind_solve_power.html

In [4]:
es = sms.proportion_effectsize(0.20, 0.25)
sms.NormalIndPower().solve_power(es, power=0.80, alpha=0.05, ratio=1)

1091.8961587171991

Тут прям вообще супер сходится. Принимая решение я бы в этом случае округлил бы до 1100. И мне бы точно хватило наблюдений для получения это разницы.

Еще можем проверить это с помощью бутстрепа.

In [5]:
n = 10000
res = []
for _ in tqdm(range(n)):
    p1 = np.random.binomial(1,0.20,1100)
    p2 = np.random.binomial(1,0.25,1100)
    cnts = [p1.sum(),p2.sum()]
    nobs = [len(p1),len(p2)]
    pval = proportions_ztest(cnts,nobs)[1]
    res.append(pval <= 0.05)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [6]:
np.mean(res)

0.8004

Здесь я посчитал % когда наш тест правильно отвергает нулевую гипотезу. Это процент называется мощность, в всех случаях мы искали такие размеры сэмпла при которых будет получена мощность 80%. Кстати, таким методом можно подбирать размеры сэмплов на глазок, если не веришь формулам).

Размеры выборок не должны быть одинаковыми. Они могут быть какими угодно. Но вот Эван Миллер не умеет считать размеры для таких случаев). Поэтому надо пользоваться чем-то другим, например питон. Сначала глянем мою функцию. Например мы хотим, чтобы второй семпл был больше в два раза, т.е. большая пилотная группа

In [7]:
get_sample_size(0.20,0.25, r = 2)

(828.557745393677, 1657.115490787354)

Сравним с statmodels

In [8]:
es = sms.proportion_effectsize(0.20, 0.25)
sms.NormalIndPower().solve_power(es, power=0.80, alpha=0.05, ratio=2)

818.9221190753085

Он возвращает только размер первой группы, вторую нужно считать самим). Почти сходится, но верим конечно же больше станмоделсам. И в любом случае лучше округлить до 900. Проверим бутсрепом.

In [9]:
n = 10000
res = []
for _ in tqdm(range(n)):

    p1 = np.random.binomial(1, 0.2, 900)
    p2 = np.random.binomial(1, 0.25, 900*2)

    cnts = [p1.sum(),p2.sum()]
    nobs = [len(p1),len(p2)]
    pval = proportions_ztest(cnts,nobs)[1]
    res.append(pval <= 0.05)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [10]:
np.mean(res)

0.8353

Ну тут мы лихо округлили, поэтому и перевалило за 80%.